<a href="https://colab.research.google.com/github/francesco-source/CDMO/blob/main/SAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install z3-solver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 MB 11.5 MB/s eta 0:00:00


In [ ]:
from itertools import combinations
from z3 import *
import math
import numpy as np

In [ ]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_least_one(bool_vars), at_most_one(bool_vars, name))



In [ ]:
m = 3
n = 7
l = [15, 10, 10]
s = [3, 2, 6, 8, 5, 4, 4]
D = [[0, 3, 3, 6, 5, 6, 6, 2],
     [ 3, 0, 4, 3, 4, 7, 7, 3],
     [3, 4, 0, 7, 6, 3, 5, 3],
     [6, 3, 7, 0, 3, 6, 6, 4],
     [5, 4, 6, 3, 0, 3, 3, 3],
     [6, 7, 3, 6, 3, 0, 2, 4],
     [6, 7, 5, 6, 3, 2, 0, 4],
     [2, 3, 3, 4, 3, 4, 4, 0]]


In [26]:
class Postmans:
    def __init__(self, m, n, l, s, D):
        self.m = m
        self.n = n
        self.l = l
        self.s = s
        self.D = D

        
instance = Postmans(m,n,l,s,D)

In [51]:
def mtps(instance):
  m = instance.m
  n = instance.n
  l = instance.l
  s = instance.s
  D = instance.D

  sol = Solver()

  constraints = []

  X = [[[Bool(f"x_{i}_{j}_{k}") for k in range(0,n+1)] for j in range(n)] for i in range(m)]

# each cell has only one value.
  for i in range(m):
    for j in range(n):
      sol.add(exactly_one(X[i][j],f"valid_cell_{i}_{j}"))

# each element except zero should be seen only once inside the matrix
  for k in range(1,n):
      sol.add(exactly_one([X[i][j][k] for i in range(m) for j in range(n)],f"valid_k{k}"))

# each courier can only deliver items whose total size doesn't exceed limit
  for i in range(m):
     sol.add(PbLe([(X[i][j][k],s[k-1]) for j in range(n) for k in range(1,n+1)], l[i]))

  





    


  if sol.check() == sat:
    sol = []
    return sol
  else:
        print("Failed to solve")
mtps(instance)
      

[]